In [4]:
!pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-10

In [ ]:
# file: train_coatnet_gender.py

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, WeightedRandomSampler
from torchvision import datasets, transforms
from torchvision.transforms import ColorJitter, RandomRotation
import timm
import numpy as np

# Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
TRAIN_DIR = 'C:\\Users\\mrinmoy\\Documents\\GitHub\\hackthoncomsys_face_classification_challenge\\Comsys_Hackathon5\\Task_A\\train'
VAL_DIR = 'C:\\Users\\mrinmoy\\Documents\\GitHub\\hackthoncomsys_face_classification_challenge\\Comsys_Hackathon5\\Task_A\\val'

# Transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    ColorJitter(0.3, 0.3, 0.3, 0.1),
    RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# Dataset and Weighted Sampler
train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=val_transforms)

class_counts = np.bincount([label for _, label in train_dataset])
weights = 1. / class_counts
sample_weights = [weights[label] for _, label in train_dataset]

sampler = WeightedRandomSampler(
    sample_weights,
    num_samples=len(sample_weights),
    replacement=True
)

train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

# Model: CoAtNet-0
class GenderClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = timm.create_model("coatnet_0_rw_224", pretrained=True, num_classes=2)

    def forward(self, x):
        return self.model(x)

model = GenderClassifier().to(DEVICE)

# Loss, Optimizer, Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Validation Function
def validate():
    model.eval()
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            correct += (outputs.argmax(1) == labels).sum().item()
    return correct / len(val_loader.dataset)

# Training Loop
def train(num_epochs=30):
    best_acc = 0
    patience = 5
    no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        scheduler.step()
        train_acc = correct / total
        val_acc = validate()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss:.4f}, "
              f"Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            no_improve = 0
            torch.save(model.state_dict(), "best_coatnet_gender_model.pth")
            print("Saved best model.")
        #else:
        #    no_improve += 1
        #    if no_improve >= patience:
        #        print("Early stopping triggered.")
        #        break

# Main
if __name__ == '__main__':
    train(num_epochs=200)


model.safetensors:   0%|          | 0.00/110M [00:00<?, ?B/s]

Epoch [1/200], Loss: 619.2568, Train Acc: 0.8525, Val Acc: 0.9052
Saved best model.
Epoch [2/200], Loss: 419.8364, Train Acc: 0.9159, Val Acc: 0.8436
Epoch [3/200], Loss: 345.0363, Train Acc: 0.9294, Val Acc: 0.9384
Saved best model.
Epoch [4/200], Loss: 302.8280, Train Acc: 0.9330, Val Acc: 0.9194
Epoch [5/200], Loss: 242.4070, Train Acc: 0.9564, Val Acc: 0.9076
Epoch [6/200], Loss: 199.4996, Train Acc: 0.9626, Val Acc: 0.9147
Epoch [7/200], Loss: 171.9358, Train Acc: 0.9631, Val Acc: 0.9218
Epoch [8/200], Loss: 154.6634, Train Acc: 0.9678, Val Acc: 0.9171
Epoch [9/200], Loss: 141.2239, Train Acc: 0.9678, Val Acc: 0.9100
Epoch [10/200], Loss: 108.7199, Train Acc: 0.9777, Val Acc: 0.9100
Epoch [11/200], Loss: 118.7705, Train Acc: 0.9772, Val Acc: 0.9123
Epoch [12/200], Loss: 117.9963, Train Acc: 0.9761, Val Acc: 0.9100
Epoch [13/200], Loss: 107.8569, Train Acc: 0.9798, Val Acc: 0.9171
Epoch [14/200], Loss: 130.0457, Train Acc: 0.9699, Val Acc: 0.9076
Epoch [15/200], Loss: 118.0820, Tra